In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#browser = webdriver.Chrome('C:\\chromedriver_win32\\chromedriver.exe')
browser = webdriver.Firefox()
# browser.set_window_size(1024, 600)
browser.maximize_window()
browser.implicitly_wait(3)

In [8]:

browser.get('https://www.hbs.edu/mba/find-answers/Pages/default.aspx')


In [9]:
count = len(browser.find_elements_by_tag_name("dt"))

In [10]:
count

75

In [11]:
#element = browser.find_elements_by_tag_name("dt")[5]

In [12]:
#browser.execute_script("arguments[0].click();", element)

qa = []

In [13]:

for i in range(count):
    element = browser.find_elements_by_tag_name("dt")[i]
    ques = element.text
    browser.execute_script("arguments[0].click();", element)
    answer = browser.find_elements_by_tag_name("dd")[i].text
    qa.append((ques,answer))
    #print("###################################################end######################################")
    
    

In [14]:
qa

[(u'+What is the timeline for applying?', u'View application deadlines.'),
 (u'+How will my application be evaluated?',
  u'Each application received is reviewed in depth by the Admissions Board.  When selecting students we put an emphasis on leadership potential, strong academic ability, and personal qualities and characteristics.  Find out more.'),
 (u'+In which part of the application may I enter additional information, including special circumstances?',
  u'Please enter important information that you want the Board to know in the Additional Information section of the application.'),
 (u'+How do I add new information if I have already submitted my application?',
  u'Once the application has been submitted, the Admissions Board is unable to accept application updates other than contact updates.  If invited to interview, applicants may inform the Board of updates at that time.'),
 (u'+Does the reputation/history of my undergraduate institution or company affect my chances of being acc

In [15]:
type(qa)

list

In [16]:
import pandas as pd

In [30]:
#print pd.DataFrame(list(qa))
df= pd.DataFrame(list(qa))
#df2=pd.DataFrame(list(qa2), columns=['Questions','Answers'])
#print pd.DataFrame(list(qa), columns=['Questions','Answers'])

In [18]:
df.to_csv('qa.csv',encoding='utf-8')

In [ ]:
print df

In [ ]:
#df2 = pd.DataFrame(list(qa))

In [31]:
df_question=df[0]
df_ans=df[1]

In [28]:
df_question
#sorted(df3)

0                   +What is the timeline for applying?
1                +How will my application be evaluated?
2     +In which part of the application may I enter ...
3     +How do I add new information if I have alread...
4     +Does the reputation/history of my undergradua...
5     +When I print the application in PDF format, p...
6     +What is the desired length of work experience...
7                    +What constitutes work experience?
8     +Are there specific types of work experience t...
9                +Who should recommendations come from?
10               +What is the process for recommenders?
11    +Is the recommendation deadline the same as th...
12    +When should I expect an invitation for an int...
13              +What should I expect for my interview?
14                             +How can I best prepare?
15    +How can I learn more about the Post Interview...
16    +Who should I contact if I have a question reg...
17    +When and how can I expect to hear back fr

In [ ]:
#df_question.to_csv('qanda.csv',sep=',',encoding='utf-8')

In [26]:
df=pd.read_csv('F:\\Download 2\\qa.csv', sep='?')

ParserError: Error tokenizing data. C error: Expected 2 fields in line 27, saw 3


In [32]:
df.head()

,0,1
0,+What is the timeline for applying?,View application deadlines.
1,+How will my application be evaluated?,Each application received is reviewed in depth...
2,+In which part of the application may I enter ...,Please enter important information that you wa...
3,+How do I add new information if I have alread...,"Once the application has been submitted, the A..."
4,+Does the reputation/history of my undergradua...,Our goal is to admit a class that offers a var...


In [33]:
import nltk
import pandas as pd
from nltk import FreqDist
from numpy import dot
from numpy.linalg import norm
from operator import add

In [34]:
sorted(df)

[0, 1]

In [36]:
df.columns = ['question', 'answer']

In [ ]:
stop_words=["what","when","how","i","do","which","is","for","a","am","not","in","on","are","or",
            "was","may","to","the","like"]

df_question=df['question']
df_answer=df['answer']


question_string=""
for question in df_question:
    question_string=question_string+" "+str(question)

fred_dict=FreqDist(nltk.tokenize.word_tokenize(question_string))

unique_words=list(fred_dict.keys())
unique_words=[word.lower() for word in unique_words]

for word in stop_words:
    if word in unique_words:
        unique_words.remove(word)

word_dict={}
count=0
for word in unique_words:
    word_dict[word]=count
    count=count+1

def createWordVector(word):
    word=word.lower()
    vec=[0]*count
    if word in unique_words:
        vec[word_dict[word]]=1
    # else:
    #     vec[count-1]=1
    return vec

def getSentenceVector(sentence):
    words = nltk.word_tokenize(sentence)
    word_vectors=[]
    # word_vectors = [createOneHot(word) for word in words]
    for word in words:
        word_vec=createWordVector(word)
        word_vectors.append(word_vec)
    sum=[0]*count
    for vec in word_vectors:
        sum=map(add,sum,vec)

    sentence_vector = [elem / len(words) for elem in sum]

    return sentence_vector

def getAnswer(sentence):
    train_sent_vectors=[]
    similarity=[]
    test_vec = getSentenceVector(sentence)

    for question in df_question:
        question_vec=getSentenceVector(question)
        train_sent_vectors.append(question_vec)
        dot_product=dot(test_vec, question_vec)
        if dot_product!=0:
            cos_sim = dot(test_vec, question_vec) / (norm(test_vec) * norm(question_vec))
        else:
            cos_sim=0
        similarity.append(cos_sim)
    max_index=similarity.index(max(similarity))

    final_answer=""
    if similarity[max_index]>0.45:
        final_answer=df_answer[max_index]
    else:
        final_answer="I am sorry. I don't have an answer for that"

    return final_answer


if __name__ == '__main__':
    question="How important is CGPA for admission?"
    answer=getAnswer(question)
    print("Question: ",question)
    print("Answer: ",answer)


In [41]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
 
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
 
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0
 
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        best_score = max([synset.path_similarity(ss) for ss in synsets2])
 
        # Check that the similarity could have been computed
        if best_score is not None:
            score += best_score
            count += 1
 
    # Average the values
    score /= count
    return score
 
sentences = [
    "What is the timeline for applying?"
]
 
focus_sentence = "How are you?"
 
for sentence in sentences:
    print "Similarity(\"%s\", \"%s\") = %s" % (focus_sentence, sentence, sentence_similarity(focus_sentence, sentence))
    print "Similarity(\"%s\", \"%s\") = %s" % (sentence, focus_sentence, sentence_similarity(sentence, focus_sentence))

Similarity("How are you?", "What is the timeline for applying?") = 1.0
Similarity("What is the timeline for applying?", "How are you?") = 0.666666666667
